<img src="images/dask_horizontal.svg" align="right" width="30%">

# Arrays

<img src="images/array.png" width="25%" align="right">
Dask array使用分块算法提供了一个并行的，大于内存的n维数组。 简而言之：分布式Numpy。

* **并行**：使用计算机上的所有内核
* **大于内存**：通过将数组分成许多小块，以最小化计算的内存占用的顺序操作，可以处理大于可用内存的数据集，并且 有效地从磁盘流式传输数据。
* **分块算法**：通过执行许多较小的计算来执行大型计算

在这个notebook中，我们将通过从头开始实现一些分块的算法，来建立一些理解。
然后，我们将使用熟悉的类似NumPy的API，使用Dask Array并行分析大型数据集。

**相关文档**

* [Array 文档](https://docs.dask.org/en/latest/array.html)
* [Array 屏幕录像](https://youtu.be/9h_61hXCDuI)
* [Array API](https://docs.dask.org/en/latest/array-api.html)
* [Array 示例](https://examples.dask.org/array.html)

## 创建数据

In [ ]:
%run prep.py -d random

## 设置

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

## 分块算法

*分块算法*通过将大型数据集分解为许多小块来在大型数据集上执行。

例如，考虑取十亿个数字的总和。 我们可以将数组分成 1,000 个块，每个块的大小为 1,000,000，取每个块的总和，然后取中间总和的总和。

我们通过执行许多较小的结果（每个一百万个数字的一千个总和，然后是另一个一千个数字的总和）来实现预期的结果（一个十亿个数字的总和）。

在以下示例中，我们使用 Python 和 NumPy 完全做到了这一点：

In [ ]:
# 使用 h5py 加载数据
# 这将创建一个指向数据的指针，但不会实际加载数据
import h5py
import os
f = h5py.File(os.path.join('data', 'random.hdf5'), mode='r')
dset = f['/x']

**使用分块算法求和**

在使用 dask 之前，让我们考虑一下阻塞算法的概念。 我们可以通过逐块加载它们并保存运行总数来计算大量元素的总和。

这里我们计算磁盘上这个大数组的总和

1. 计算数组中每个 1,000,000 大小块的总和
2. 计算 1,000 个中间和的总和

请注意，这是notebook内核中的一个顺序过程，包括加载和求和。

In [ ]:
# 计算大型数组的总和，每次计算一百万个数字
sums = []
for i in range(0, 1_000_000_000, 1_000_000):
    chunk = dset[i: i + 1_000_000]  # 取出numpy数组
    sums.append(chunk.sum())

total = sum(sums)
print(total)

### 练习：使用分块算法计算均值

现在，我们已经看到了上面的简单示例，尝试做一个稍微复杂一些的问题，计算数组的均值，假设我们暂时不知道数据中有多少个元素。 您可以通过以下代码更改上面的代码来做到这一点：

1. 计算每个块的总和
2. 计算每个块的长度
3. 计算1,000个中间总和的总和与1,000个中间长度的总和，然后相除。

这种方法对于我们的案例来说有点矫枉过正，但如果我们事先不知道数组或单个块的大小，这种方法却很具有泛化性。

In [ ]:
# 计算数组的均值

In [ ]:
sums = []
lengths = []
for i in range(0, 1_000_000_000, 1_000_000):
    chunk = dset[i: i + 1_000_000]  # 取出numpy数组
    sums.append(chunk.sum())
    lengths.append(len(chunk))

total = sum(sums)
length = sum(lengths)
print(total / length)

`dask.array` 包含这些算法
-----------------------------------------

Dask.array 是一个类似 NumPy 的库，它执行这些技巧来对不适合内存的大型数据集进行操作。 它超越了上面讨论的线性问题，扩展到完整的 N 维算法，是NumPy 接口的一个不错的子集。

**创建`dask.array`对象**

您可以使用 `da.from_array` 函数创建一个 `dask.array` `Array` 对象。 该函数接受

1. `data`：任何支持 NumPy 切片的对象，如 `dset`
2. `chunks`：告诉我们如何阻塞数组的块大小，例如 `(1_000_000,)`

In [ ]:
import dask.array as da
x = da.from_array(dset, chunks=(1_000_000,))
x

**像处理 numpy 数组一样操作 `dask.array` 对象**

现在我们有了一个 `Array`，我们可以执行标准的 numpy 风格的计算，比如算术、数学、切片、归约等。

端口是熟悉的，但实际工作却有所不同。 `dask_array.sum()` 与 `numpy_array.sum()` 的作用不同。

**有什么不同?**

`dask_array.sum()`建立一个计算表达式。 它尚未执行计算。 `numpy_array.sum()`立即计算总和。

*为什么有区别？*

Dask 数组被分成块。 每个块都必须显式地在该块上运行计算。 如果所需的答案来自整个数据集的一小部分，则在所有数据上运行计算将浪费 CPU 和内存。

In [ ]:
result = x.sum()
result

**计算结果**

Dask.array 对象被惰性地评估。 像 `.sum` 这样的操作会构建一个要执行的阻塞任务图。

我们通过调用 `.compute()` 来请求最终结果。 这会触发实际的计算。

In [ ]:
result.compute()

### 练习：计算均值

以及方差，标准差等。这应该是对上面示例的一个很小的更改。

查看使用Jupyter notebook的Tab键补全可以执行的其他操作。

这与您之前的结果相符吗？

性能和并行性
-------------------------------

<img src="images/fail-case.gif" width="40%" align="right">

在我们的第一个例子中，我们使用 `for` 循环一次一个块地遍历数组。 对于像`sum`这样的简单操作，这是最佳的。 然而，对于复杂的操作，我们可能希望以不同的方式遍历数组。 特别是我们可能需要以下内容：

1. 多核并行使用
2. 在进入下一个区块之前在单个区块上进行链式操作

`Dask.array` 将您的数组操作转换为相互关联的任务图，它们之间具有数据依赖关系。 然后 Dask 与多个线程并行执行此图。 我们将在下一节中详细讨论这一点。

### 示例

1. 构造一个20000x20000的正态分布随机值数组，将其分解为1000x1000个大小的块
2. 沿一条轴取平均值
3. 每隔100个元素取值

In [ ]:
import numpy as np
import dask.array as da

x = da.random.normal(10, 0.1, size=(20000, 20000),   # 4亿个元素的数组 
                              chunks=(1000, 1000))   # 切成 1000x1000 的块
y = x.mean(axis=0)[::100]                            # 进行 NumPy风格的操作

In [ ]:
x.nbytes / 1e9  # 数千兆字节的输入被延迟处理了

In [ ]:
%%time
y.compute()     # 计算结果的时间

性能对比
---------------------------

以下实验是在重型个人笔记本电脑上进行的。 您机器的表现可能会有所不同。 如果您尝试使用 NumPy 版本，请确保您拥有超过 4GB 的内存。

**NumPy: 19s, 需要数千兆的内存**

```python
import numpy as np

%%time 
x = np.random.normal(10, 0.1, size=(20000, 20000)) 
y = x.mean(axis=0)[::100] 
y

CPU times: user 19.6 s, sys: 160 ms, total: 19.8 s
Wall time: 19.7 s
```

**Dask Array: 4s, 需要数兆内存**

```python
import dask.array as da

%%time
x = da.random.normal(10, 0.1, size=(20000, 20000), chunks=(1000, 1000))
y = x.mean(axis=0)[::100] 
y.compute() 

CPU times: user 29.4 s, sys: 1.07 s, total: 30.5 s
Wall time: 4.01 s
```

**讨论**

请注意，Dask 数组计算运行了 4 秒，但使用了 29.4 秒的用户 CPU 时间。 numpy 计算运行了 19.7 秒，并使用了 19.6 秒的用户 CPU 时间。

Dask 完成得更快，但使用了更多的总 CPU 时间，因为 Dask 能够因为块大小而在用户无感知的状态下进行并行化计算。

*问题*

*  如果 dask chunks=(20000,20000)，会发生什么?
    * 计算是否会在4s内完成?
    * 需要消耗多少内存?
*  如果dask chunks=(25,25)，会发生什么?
    * CPU和内存占用会如何变化？

### 练习：气象数据

在`data/weather-big/*`的hdf5文件中有2GB的人造气象数据。 我们将使用`h5py`库与该数据进行交互，并使用`dask.array`对其进行计算。

我们的目标是可视化本月地球表面的平均温度。 这将需要对所有这些数据进行平均。 我们将按照以下步骤进行操作

1. 为磁盘上的每一天数据创建`h5py.Dataset`对象（`dsets`）
2. 用`da.from_array`调用包装它们
3. 通过调用`da.stack`随时间堆叠这些数据集
4. 使用`.mean()`方法计算沿新堆积的时间轴的平均值
5. 使用`matplotlib.pyplot.imshow`可视化结果

In [ ]:
%run prep.py -d weather

In [ ]:
import h5py
from glob import glob
import os

filenames = sorted(glob(os.path.join('data', 'weather-big', '*.hdf5')))
dsets = [h5py.File(filename, mode='r')['/t2m'] for filename in filenames]
dsets[0]

In [ ]:
dsets[0][:5, :5]  # 切片为h5py.Dataset对象会得到一个numpy数组

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 8))
plt.imshow(dsets[0][::4, ::4], cmap='RdBu_r');

**与`dask.array`集成**

使用chuck大小为`(500,500)`的`da.from_array`函数，从`h5py.Dataset`对象列表中列出`dask.array`对象列表。

In [ ]:
arrays = [da.from_array(dset, chunks=(500, 500)) for dset in dsets]
arrays

**将这个数组`dask.array`对象堆叠到一个带有`da.stack`的`dask.array`对象中**

将它们沿着第一条轴堆叠，以使结果数组的形状为`(31,5760,11520)`。

In [ ]:
x = da.stack(arrays, axis=0)
x

**沿时间`(第0)`轴绘制此数组的平均值**

In [ ]:
# 完成以下：
fig = plt.figure(figsize=(16, 8))
plt.imshow(..., cmap='RdBu_r')

In [ ]:
result = x.mean(axis=0)
fig = plt.figure(figsize=(16, 8))
plt.imshow(result, cmap='RdBu_r');

**绘制第一天与平均值的差异** 

In [ ]:
result = x[0] - x.mean(axis=0)
fig = plt.figure(figsize=(16, 8))
plt.imshow(result, cmap='RdBu_r');

### 练习：子采样和存储

在上面的练习中，我们的计算结果很小，因此我们可以安全地调用`compute`。 有时我们的结果仍然太大而无法容纳到内存中，因此我们想将其保存到磁盘中。 在这些情况下，您可以使用以下两个功能之一

1.  `da.store`: 将`dask.array`存储到任何支持numpy setitem语法的对象中, 例如.

        f = h5py.File('myfile.hdf5')
        output = f.create_dataset(shape=..., dtype=...)
        
        da.store(my_dask_array, output)
        
2.  `da.to_hdf5`: 一种特制的函数，用于创建dask.array对象并将其存储到HDF5文件中。

        da.to_hdf5('data/myfile.hdf5', '/output', my_dask_array)
        
本练习中的任务是**使用numpy步骤切片在纬度和经度方向上对整个数据集进行二次抽样，然后再使用上面列出的功能之一将此结果存储到磁盘**。

提醒一下，Python切片需要三个要素

    start:stop:step

    >>> L = [1, 2, 3, 4, 5, 6, 7]
    >>> L[::3]
    [1, 4, 7]

In [ ]:
# ...

In [ ]:
import h5py
from glob import glob
import os
import dask.array as da

filenames = sorted(glob(os.path.join('data', 'weather-big', '*.hdf5')))
dsets = [h5py.File(filename, mode='r')['/t2m'] for filename in filenames]

arrays = [da.from_array(dset, chunks=(500, 500)) for dset in dsets]

x = da.stack(arrays, axis=0)

result = x[:, ::2, ::2]

# 译者注：zarr是一种Python 的分块、压缩、N 维数组的实现。详见https://zarr.readthedocs.io/en/stable/
# 其和hdf文件有点类似，即一个文件里面可以包含很多不同的dataset
# 与hdf文件不同，直观上看hdf是一个单一的文件，而zarr是一个文件夹，里面还包含不同的子文件夹（相当于hdf中的dataset），子文件夹下面保存着数据
# 相对于hdf文件，zarr的读写速度更快。并且，其特别适合用于数组在s3等对象存储上的保存。
da.to_zarr(result, os.path.join('data', 'myfile.zarr'), overwrite=True)

## 示例：兰纳-琼斯势

[兰纳-琼斯势](https://baike.baidu.com/item/%E5%85%B0%E7%BA%B3-%E7%90%BC%E6%96%AF%E5%8A%BF)被用于物理，化学和工程领域的粒子模拟。 它是高度可并行化的。
首先，我们将对7,000个粒子运行和配置Numpy版本。

译者注：兰纳-琼斯势（Lennard-Jones potential），又称L-J势, 6-12势, 或12-6势，是用来模拟两个电中性的分子或原子间相互作用势能的一个比较简单的数学模型。最早由数学家约翰·兰纳-琼斯于1924年提出。由于其解析形式简单而被广泛使用，特别是用来描述惰性气体分子间相互作用尤为精确。

In [ ]:
import numpy as np

# 创建一个粒子的随机集合 
def make_cluster(natoms, radius=40, seed=1981):
    np.random.seed(seed)
    cluster = np.random.normal(0, radius, (natoms,3))-0.5
    return cluster

def lj(r2):
    sr6 = (1./r2)**3
    pot = 4.*(sr6*sr6 - sr6)
    return pot

# 建立距离矩阵
def distances(cluster):
    diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
    mat = (diff*diff).sum(-1)
    return mat

# 在移除接近零的距离后，lj 函数在上层三角形上进行评估
def potential(cluster):
    d2 = distances(cluster)
    dtri = np.triu(d2)
    energy = lj(dtri[dtri > 1e-6]).sum()
    return energy

In [ ]:
cluster = make_cluster(int(7e3), radius=500)

In [ ]:
%time potential(cluster)

注意，最花时间的函数是 `distances`:

In [ ]:
# 这将在另一个浏览器标签中打开
# %load_ext snakeviz
# %snakeviz potential(cluster)

# 在此选项卡中给出文本结果的替代简单版本
%prun -s tottime potential(cluster)

### Dask 版本

这是 Dask 的版本，只有`potential`函数需要重写,以最好地利用 Dask。

注意，`da.nansum` 已经在完整的 $NxN$ 距离矩阵上使用，以提高并行效率。

In [ ]:
import dask.array as da

# 计算整个距离矩阵的势
# 并忽略除以零
def potential_dask(cluster):
    d2 = distances(cluster)
    energy = da.nansum(lj(d2))/2.
    return energy

让我们将 NumPy 数组转换为 Dask 数组。由于整个 NumPy 数组可以安装在内存中，所以按照 CPU 核心数量来分块数组更加有效。

In [ ]:
from os import cpu_count

dcluster = da.from_array(cluster, chunks=cluster.shape[0]//cpu_count())

这一步应该可以很好地扩展内核数量。 会出现除以零(`divide by zero`)的警告(`RuntimeWarning`)，这就是我们在 `potential_dask` 中使用 `da.nansum` 的原因。

In [ ]:
e = potential_dask(dcluster)
%time e.compute()

局限性
-----------

Dask Array 没有实现整个 numpy 接口。期待这一点的用户会感到失望。值得注意的是 Dask Array 有以下缺点：

1. Dask 没有实现所有的``np.linalg``。这已经由许多优秀的 BLAS/LAPACK 实现完成，并且是众多正在进行的学术研究项目的重点。
2. Dask Array 不支持某些结果形状取决于数组值的操作。对于它确实支持的那些（例如，用另一个布尔掩码屏蔽一个 Dask 数组），块大小将是未知的，这可能会导致其他需要知道块大小的操作出现问题。
3. Dask Array 不会尝试像"排序"(``sort``)这样的操作，众所周知，这些操作很难并行执行，并且在非常大的数据上价值会有所降低（您实际上很少需要完整排序）。
   通常我们包括并行友好的替代方案，如“topk”。
4. Dask 开发是由迫在眉睫的需求驱动的，所以很多不太常用的功能，比如 ``np.sometrue`` 纯粹由于我们的懒惰而没有实现。这些将由做出出色的社区贡献。
    
* [Array 文档](https://docs.dask.org/en/latest/array.html)
* [Array 屏幕录像](https://youtu.be/9h_61hXCDuI)
* [Array API](https://docs.dask.org/en/latest/array-api.html)
* [Array 示例](https://examples.dask.org/array.html)

In [ ]:
client.shutdown()